In [ ]:
import pandas as pd
import requests

# URL pour les données des communes de l'INSEE
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Charger le fichier CSV contenant la liste des communes
df = pd.read_csv(url, delimiter=";")  # Le séparateur est probablement ';' pour les fichiers français
print("Fichier chargé avec succès !")
print(df.head())

# URL de base pour l'API Geo-DVF
geodvf_api_url = "https://files.data.gouv.fr/geo-dvf/latest/"

# Fonction pour appeler l'API Geo-DVF et obtenir les données de prix pour une commune
def get_price_data(commune_code):
    request_url = f"{geodvf_api_url}?code_commune={commune_code}&page=1&page_size=10"
    response = requests.get(request_url)
    if response.status_code == 200:
        json_data = response.json()
        if json_data and 'data' in json_data and len(json_data['data']) > 0:
            price_data = pd.DataFrame(json_data['data'])
            return price_data
    return None

# Liste pour stocker les DataFrames individuels
all_price_data = []

# Boucle pour récupérer les données pour chaque commune
for index, row in df.iterrows():
    commune_code = row['COM']  # Remplacez par la colonne contenant le code INSEE si différent
    price_data = get_price_data(commune_code)
    if price_data is not None:
        all_price_data.append(price_data)

# Combiner tous les DataFrames en un seul
if all_price_data:
    combined_price_data = pd.concat(all_price_data, ignore_index=True)
    print("Données combinées des prix :")
    print(combined_price_data.head())
    # Enregistrer dans un fichier CSV
    combined_price_data.to_csv("price_data.csv", index=False)
    print("Données sauvegardées dans 'price_data.csv'")
else:
    print("Aucune donnée valide trouvée pour les communes.")


In [2]:
import pandas as pd
import requests

# URL du fichier CSV INSEE
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Chargement du fichier CSV avec gestion des délimiteurs
try:
    df = pd.read_csv(url, sep=',', encoding='utf-8')  # On suppose un séparateur par virgule
    df.columns = df.columns.str.strip()  # Nettoyage des colonnes
    print("Fichier des communes chargé avec succès !")
    print("Colonnes disponibles :", df.columns)
except Exception as e:
    print(f"Erreur lors du chargement du fichier CSV : {e}")
    exit()

# Vérification de la colonne 'COM'
if 'COM' not in df.columns:
    print("Erreur : La colonne 'COM' n'existe pas.")
    print("Colonnes disponibles :", df.columns)
    exit()

# Exemple de traitement
for index, row in df.iterrows():
    commune_code = row['COM']  # Accéder à la colonne des codes INSEE
    print(f"Code commune : {commune_code}")
    # Ajouter le reste du traitement ici

# URL de base pour l'API DVF
dvf_api_url = "http://api.cquest.org/dvf"  # API publique pour récupérer les données DVF (Demandes de Valeurs Foncières)

# Définition d'une fonction pour récupérer les données DVF via l'API
def get_dvf_data(code_commune, nature_mutation=None, type_local=None):
    """
    Cette fonction effectue une requête HTTP à l'API DVF pour récupérer les données
    correspondant à une commune donnée, avec des filtres optionnels pour la nature
    de la mutation (ex: Vente) et le type de local (ex: Maison).

    :param code_commune: Code INSEE de la commune (obligatoire)
    :param nature_mutation: Type de mutation, comme 'Vente' (optionnel)
    :param type_local: Type de bien immobilier, comme 'Maison' (optionnel)
    :return: Un DataFrame pandas contenant les données DVF ou None si aucune donnée
    """
    # Préparation des paramètres pour la requête API
    params = {
        "code_commune": code_commune  # Code INSEE de la commune, obligatoire
    }
    # Ajouter le filtre pour la nature de mutation si fourni
    if nature_mutation:
        params["nature_mutation"] = nature_mutation
    # Ajouter le filtre pour le type de local si fourni
    if type_local:
        params["type_local"] = type_local

    # Envoyer la requête GET à l'API avec les paramètres spécifiés
    response = requests.get(dvf_api_url, params=params)

    # Vérifier si la réponse de l'API est réussie (statut HTTP 200)
    if response.status_code == 200:
        # Convertir la réponse JSON en dictionnaire Python
        json_data = response.json()
        # Vérifier si des résultats sont présents dans la clé 'features'
        if json_data and 'features' in json_data:
            # Extraire les propriétés des résultats, qui contiennent les données pertinentes
            data = [feature['properties'] for feature in json_data['features']]
            # Convertir les données en DataFrame pandas si des données existent
            return pd.DataFrame(data) if data else None
    return None  # Retourner None si aucune donnée n'a été trouvée ou si la requête a échoué

# Créer une liste vide pour stocker les DataFrames individuels pour chaque commune
all_dvf_data = []

# Boucler sur chaque ligne du DataFrame des communes pour récupérer leurs données
for index, row in df.iterrows():
    """
    Cette boucle parcourt chaque ligne du fichier INSEE contenant les données des communes.
    À chaque itération, elle extrait le code INSEE de la commune et effectue une requête
    à l'API DVF pour obtenir les données de la commune.
    """
    code_commune = row['COM']  # Récupérer le code INSEE de la commune depuis la colonne 'COM'

    # Appeler la fonction pour récupérer les données DVF avec des filtres
    # Exemple : On filtre ici pour récupérer uniquement les ventes de maisons
    dvf_data = get_dvf_data(code_commune, nature_mutation="Vente", type_local="Maison")
    
    # Si des données sont retournées, les ajouter à la liste
    if dvf_data is not None:
        all_dvf_data.append(dvf_data)

# Une fois toutes les communes traitées, combiner les résultats en un seul DataFrame
if all_dvf_data:
    # Combiner tous les DataFrames individuels en un seul DataFrame
    combined_dvf_data = pd.concat(all_dvf_data, ignore_index=True)
    print("Données DVF combinées :")  # Message de confirmation
    print(combined_dvf_data.head())  # Afficher un aperçu des données combinées

    # Sauvegarder les données combinées dans un fichier CSV
    combined_dvf_data.to_csv("dvf_data.csv", index=False)  # Sauvegarde sans inclure les index
    print("Données sauvegardées dans 'dvf_data.csv'")  # Confirmation de la sauvegarde
else:
    # Si aucune donnée n'a été trouvée pour toutes les communes
    print("Aucune donnée valide trouvée pour les communes.")


Fichier des communes chargé avec succès !
Colonnes disponibles : Index(['TYPECOM', 'COM', 'REG', 'DEP', 'CTCD', 'ARR', 'TNCC', 'NCC', 'NCCENR',
       'LIBELLE', 'CAN', 'COMPARENT'],
      dtype='object')
Code commune : 01001
Code commune : 01002
Code commune : 01004
Code commune : 01005
Code commune : 01006
Code commune : 01007
Code commune : 01008
Code commune : 01009
Code commune : 01010
Code commune : 01011
Code commune : 01012
Code commune : 01013
Code commune : 01014
Code commune : 01015
Code commune : 01015
Code commune : 01016
Code commune : 01017
Code commune : 01019
Code commune : 01021
Code commune : 01022
Code commune : 01023
Code commune : 01024
Code commune : 01025
Code commune : 01025
Code commune : 01026
Code commune : 01027
Code commune : 01028
Code commune : 01029
Code commune : 01030
Code commune : 01031
Code commune : 01032
Code commune : 01033
Code commune : 01033
Code commune : 01034
Code commune : 01035
Code commune : 01036
Code commune : 01036
Code commune : 010